<a href="https://colab.research.google.com/github/qianzhou1982/Demo/blob/master/D%C3%A9corateurs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Décorateurs

Un décorateur est une fonction qui manipule une fonction ou une classe. C'est un élément du langage Python très pratique.

 Nous allons voir dans ces travaux pratiques comment les utiliser.

## Décorateurs de fonctions : fonctions qui manipulent des fonctions

Écrivez une fonction `squared`, qui prend en entrée une fonction et qui renvoie cette même fonction dont le résultat est passé au carré :

- la fonction originale prend toujours deux arguments
- la fonction originale produit toujours un nombre
- la fonction squared prend en entrée et produit en sortie **une fonction**

In [ ]:
def add(a, b):
  return a + b


def sub(a, b):
  return a - b


def mul(a, b):
  return a * b


def squared(function):
  return function  # Votre code ici


print(squared(add)(2, 2))  # Doit valoir 16
print(squared(sub)(10, 5))  # Doit valoir 25
print(squared(mul)(3, 4))  # Doit valoir 144

### Solution

In [ ]:
def squared(function):
  def new_function(a, b):
    return function(a, b) ** 2
  return new_function


print(squared(add)(2, 2))
print(squared(sub)(10, 5))
print(squared(mul)(3, 4))

## Syntaxe de décorateur

Pour appliquer un décorateur à une fonction, on peut utiliser la syntaxe suivante :

In [ ]:
@squared
def div(a, b):
  return a / b


print(div(8, 4))

Sans cette syntaxe qui utilise `@`, comment arriver au même résultat ?

In [ ]:
# Votre code ici

### Solution

In [ ]:
def div(a, b):
    return a / b


div = squared(div)


print(div(8, 4))

## Mesure du temps d'exécution d'une fonction

Créez un décorateur `timeit` qui affiche le temps d'exécution d'une fonction. Vous pourrez vous aider de la fonction [`time.time`](https://docs.python.org/fr/3/library/time.html#time.time)

In [ ]:
# Votre code ici

### Solution

In [ ]:
import time


def timeit(f):
  def new_f(*args, **kwargs):
    start = time.time()
    result = f(*args, **kwargs)
    print(f"Time elapsed during '{f.__name__}' call: {time.time() - start}s")
    return result
  return new_f


@timeit
def add(a, b):
  return a + b


add(1, 2)

## Décorateur avec arguments

Des fois, il est intéressant de pouvoir donner des arguments à un décorateur.

Pour cela, il est nécessaire de créer une "couche" supplémentaire de fonction : on va définir une fonction avec des arguments, qui renverra un décorateur sans arguments (donc lui-même une fonction qui prend en argument une fonction… ce n'est pas facile).

Voyez l'exemple suivant, qui ajoute au décorateur `timeit` vu précedemment un argument pour afficher en secondes ou millisecondes le résultat :

In [ ]:
import time


def timeit(unit="s"):
  def decorator(function):
    def new_f(*args, **kwargs):
      start = time.time()
      result = function(*args, **kwargs)
      duration = time.time() - start
      if unit == "ms":
        duration *= 1000
      elif unit != "s":
        raise ValueError("Can only use s or ms as unit argument")
      print(f"Time elapsed during '{function.__name__}' call: "
            f"{duration}{unit}")
      return result
    return new_f
  return decorator



@timeit(unit="ms")
def add(a, b):
  return a + b


add(1, 2)

## Logging automatique

Sur un schéma similaire, loggez automatiquement les appels à une fonction, avec les arguments utilisés. Vous pourrez utiliser la fonction [`print`](https://docs.python.org/fr/3/library/functions.html#print) ou les fonctions du module [`logging`](https://docs.python.org/fr/3/library/logging.html)

In [ ]:
# Votre code ici

### Solution

In [ ]:
import logging


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def autolog(logger):
  def decorator(f):
    def new_f(*args, **kwargs):
      result = f(*args, **kwargs)
      logger.info(f"%s called with pos args %s and kwargs %s",
                  f.__name__,
                  args,
                  kwargs)
      return result
    return new_f
  return decorator


@autolog(logger)
def add(a, b):
  return a + b


add(1, b=2)

## Enregistreur de classe

Il est souvent intéressant d'enregistrer des classes ou fonctions dans une structure de données (un registre), pour par exemple proposer un système de plugins.

Créez un décorateur de classe qui enregistre chaque classe à laquelle il est appliqué dans un dictionnaire, sous le nom qu'on lui donne en argument.

In [ ]:
# Votre code ici

En plus d'ajouter le décorateur aux classes que vous souhaitez enregistrer, quelle étape importante est nécessaire pour que l'enregistrement se fasse ?

*Votre réponse*

### Solution

In [ ]:
from typing import Any, Callable, Type


_plugins = {}


def register(name: str) -> Callable[[Type[Any]], Type[Any]]:
  def decorator(cls: Type[Any]) -> Type[Any]:
    _plugins[name] = cls
    return cls
  return decorator


@register("A")
class A:
  pass

Il est nécessaire de charger le module qui contient la classe. Cela peut être fait automatiquement pour rendre les systèmes de plugins plus faciles à utiliser, comme par exemple dans la librairie AllenNLP, qui a une fonction pour charger tous les modules d'un paquet donné&nbsp;: [`import_module_and_submodules`](https://github.com/allenai/allennlp/blob/a0edfae9ca571ed7d43749974bb842167201c2da/allennlp/common/util.py#L331)

## Utilisation de [`functools.wraps`](https://docs.python.org/fr/3/library/functools.html#functools.wraps)

Essayez de manipuler les fonctions modifiées dans le reste du TP et de trouver leur nom (attribut `__name__`) ou encore leur doc (attribut `__doc__`). Que remarquez-vous ?

*Votre réponse*

Utilisez `functools.wraps` sur l'exemple donné du décorateur `timeit` avec argument pour régler ce problème.

In [ ]:
# Votre code ici

### Solution

In [ ]:
import functools
import time


def timeit(unit="s"):
  def decorator(function):
    @functools.wraps(function)
    def new_f(*args, **kwargs):
      start = time.time()
      result = function(*args, **kwargs)
      duration = time.time() - start
      if unit == "ms":
        duration *= 1000
      elif unit != "s":
        raise ValueError("Can only use s or ms as unit argument")
      print(f"Time elapsed during '{function.__name__}' call: "
            f"{duration}{unit}")
      return result
    return new_f
  return decorator


@timeit(unit="ms")
def add(a, b):
  """Add two integers."""
  return a + b


print(add.__name__, add.__doc__)